In [1]:
!pip install polars
!pip install recbole

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.0 MB/s eta 0:00:00
  Attempting uninstall: colorlog
    Found existing installation: colorlog 6.7.0
    Uninstalling colorlog-6.7.0:
      Successfully uninstalled colorlog-6.7.0
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.5
    Uninstalling colorama-0.4.5:
      Successfully uninstalled colorama-0.4.5


In [2]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.utils.case_study import full_sort_topk
from typing import List, Tuple
import numpy as np
import pandas as pd
from collections import defaultdict
import torch
from pydantic import BaseModel
from recbole.data import create_dataset
from recbole.data.dataset.sequential_dataset import SequentialDataset
import polars as pl
from recbole.data.interaction import Interaction
from recbole.model.sequential_recommender.sine import SINE
from recbole.utils import get_model, init_seed


class ItemHistory(BaseModel):
    sequence: List[str]
    topk: int

class RecommendedItems(BaseModel):
    score_list: List[float]
    item_list: List[str]




In [3]:
parameter_dict = {
    'data_path': '/kaggle/input/otto-prepared-data',
    'USER_ID_FIELD': 'session',
    'ITEM_ID_FIELD': 'aid',
    'TIME_FIELD': 'ts',
    'user_inter_num_interval': "[5,Inf)",
    'item_inter_num_interval': "[5,Inf)",
    'load_col': {'inter': ['session', 'aid', 'ts']},
    'train_neg_sample_args': None,
    'epochs': 10,
    'stopping_step':3,
    'eval_batch_size': 1024,
    'MAX_ITEM_LIST_LENGTH': 20,
    'eval_args': {
        'split': {'RS': [9, 1, 0]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}


In [4]:
config = Config(model='GRU4Rec', dataset='recbox_data', config_dict=parameter_dict)
init_logger(config)
logger = getLogger()

c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

logger.info(config)


General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = /kaggle/input/otto-prepared-data/recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 3
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [9, 1, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 1024
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_s

In [5]:
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)
model = GRU4Rec(config, train_data.dataset).to(config['device'])
trainer = Trainer(config, model)


[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Evaluation]: eval_batch_size = [1024] eval_args: [{'split': {'RS': [9, 1, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}]


In [6]:
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)

epoch 0 training [time: 392.02s, train loss: 32592.3293]
epoch 0 evaluating [time: 63.49s, valid_score: 0.122200]
valid result: 
recall@10 : 0.2312    mrr@10 : 0.1222    ndcg@10 : 0.1479    hit@10 : 0.2312    precision@10 : 0.0231
Saving current: saved/GRU4Rec-Dec-28-2022_06-20-57.pth
epoch 1 training [time: 388.30s, train loss: 23811.6563]
epoch 1 evaluating [time: 62.56s, valid_score: 0.216100]
valid result: 
recall@10 : 0.3663    mrr@10 : 0.2161    ndcg@10 : 0.2517    hit@10 : 0.3663    precision@10 : 0.0366
Saving current: saved/GRU4Rec-Dec-28-2022_06-20-57.pth
epoch 2 training [time: 389.43s, train loss: 20838.2899]
epoch 2 evaluating [time: 63.79s, valid_score: 0.258000]
valid result: 
recall@10 : 0.4201    mrr@10 : 0.258    ndcg@10 : 0.2966    hit@10 : 0.4201    precision@10 : 0.042
Saving current: saved/GRU4Rec-Dec-28-2022_06-20-57.pth
epoch 3 training [time: 388.26s, train loss: 19450.2636]
epoch 3 evaluating [time: 62.90s, valid_score: 0.277700]
valid result: 
recall@10 : 0.4

In [7]:
del train_data, valid_data, test_data


In [8]:
def pred_user_to_item(item_history: ItemHistory):
    item_history_dict = item_history.dict()
    item_sequence = item_history_dict["sequence"]
    item_length = len(item_sequence)
    pad_length = MAX_ITEM  # pre-defined by recbole

    padded_item_sequence = torch.nn.functional.pad(
        torch.tensor(dataset.token2id(dataset.iid_field, item_sequence)),
        (0, pad_length - item_length),
        "constant",
        0,
    )

    input_interaction = Interaction(
        {
            "aid_list": padded_item_sequence.reshape(1, -1),
            "item_length": torch.tensor([item_length]),
        }
    )
    scores = model.full_sort_predict(input_interaction.to(model.device))
    scores = scores.view(-1, dataset.item_num)
    scores[:, 0] = -np.inf  # pad item score -> -inf
    topk_score, topk_iid_list = torch.topk(scores, item_history_dict["topk"])

    predicted_score_list = topk_score.tolist()[0]
    predicted_item_list = dataset.id2token(
        dataset.iid_field, topk_iid_list.tolist()
    ).tolist()

    recommended_items = {
        "score_list": predicted_score_list,
        "item_list": predicted_item_list,
    }
    return recommended_items


In [9]:
test = pl.read_parquet('../input/otto-full-optimized-memory-footprint/test.parquet')
session_types = ['clicks', 'carts', 'orders']
test_session_AIDs = test.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
test_session_types = test.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)
del test


In [10]:
labels = []

type_weight_multipliers = {0: 1, 1: 6, 2: 3}
for AIDs, types in zip(test_session_AIDs, test_session_types):
    if len(AIDs) >= 20:
        weights=np.logspace(0.1,1,len(AIDs),base=2, endpoint=True)-1
        aids_temp=defaultdict(lambda: 0)
        for aid,w,t in zip(AIDs,weights,types): 
            aids_temp[aid]+= w * type_weight_multipliers[t]
            
        sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
        labels.append(sorted_aids[:20])
    else:
        AIDs = list(dict.fromkeys(AIDs))
        item = ItemHistory(sequence=AIDs, topk=20)
        try:
            nns = [ int(v) for v in pred_user_to_item(item)['item_list']]
        except:
            nns = []

        for word in nns:
            if len(AIDs) == 20:
                break
            if int(word) not in AIDs:
                AIDs.append(word)

        labels.append(AIDs[:20])


In [11]:
labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]
predictions = pd.DataFrame(data={'session_type': test_session_AIDs.index, 'labels': labels_as_strings})
labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]

predictions = pd.DataFrame(data={'session_type': test_session_AIDs.index, 'labels': labels_as_strings})

prediction_dfs = []

for st in session_types:
    modified_predictions = predictions.copy()
    modified_predictions.session_type = modified_predictions.session_type.astype('str') + f'_{st}'
    prediction_dfs.append(modified_predictions)

submission = pd.concat(prediction_dfs).reset_index(drop=True)
submission.to_csv('submission.csv', index=False)